In [0]:
df = spark.table("workspace.crimestatics.total_reported_offences_1950_2023")
display(df)

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.functions import col
from functools import reduce
from pyspark.sql.types import IntegerType, FloatType, DoubleType

# load the data
df = spark.table("workspace.crimestatics.total_reported_offences_1950_2023")

# check missing values
missing_values = df.select(
    [F.sum(col(noVal).isNull().cast("int")).alias(noVal) for noVal in df.columns]
)

# display all rows with missing values
rows_with_missing_values = df.filter(
    reduce(lambda a, b: a | b, [col(c).isNull() for c in df.columns])
)

# Total number of crimes >= sum of other specific categories
category_cols = df.columns[2:] 
df = df.withColumn('sum_categories', sum([col(c) for c in category_cols]))
inconsistency = df.filter(col('Total number of crimes') < col('sum_categories'))

# Duplicates
duplicates = df.groupBy(df.columns).count().filter("count > 1")
num_duplicates = duplicates.count()

# Range checks (no negative numbers)
numeric_cols = [c for c in df.columns if df.schema[c].dataType in [IntegerType(), FloatType(), DoubleType()]]
if numeric_cols:
    negative_values = df.filter(reduce(lambda a, b: a | b, [col(c) < 0 for c in numeric_cols]))
else:
    negative_values = spark.createDataFrame([], df.schema)

print("Missing Values")
display(missing_values)
print("Rows with missing values")
display(rows_with_missing_values)
print("Inconsistency")
display(inconsistency)
print(f"Number of duplicate rows: {num_duplicates}")
print("Negative Values")
display(negative_values)